In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


### Extract all category names available, store in "cat_names"

In [25]:
url="https://blackwells.co.uk/bookshop/category/_top"
response = requests.get(url)

# Send an HTTP request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)

if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Find all HTML elements that contain developer names
    category = soup.find("ul", attrs= {'class' : 'block-filter__list category-list'})
    # Extract all "li" elements under the found "ul" element
    cat_list = category.find_all("li")

    # Create a list to store the text content of each "li" element
    book_cats = [cat.text.strip() for cat in cat_list]

    # Print the list of "li" texts
    print(book_cats)
else:
    print(f"Failed to retrieve content. Status Code: {response.status_code}")


['Art and Design', 'Biography', 'Business, Economics and Law', "Children's", 'Computing', 'Crime and Thrillers', 'Education', 'Fiction', 'Food and Drink', 'Graphic Novels', 'Health and Wellbeing', 'History', 'Home and Garden', 'Languages and Reference', 'Literature and Poetry', 'Medical and Nursing', 'Music', 'Philosophy and Religion', 'Politics', 'Science and Nature', 'SF, Fantasy & Horror', 'Social Sciences', 'Sports and Hobbies', 'Stage and Screen', 'Stationery and Gift', 'Technology and Transport', 'Teen and YA', 'Travel']


In [26]:
#Adjust name to align with url structure:
for i in range(len(book_cats)):
    if book_cats[i] == 'Crime and Thrillers':
        book_cats[i] = 'Crime'
    elif book_cats[i] == 'Science and Nature':
        book_cats[i] = 'Science'
    elif book_cats[i] == 'Business,Economics and Law':
        book_cats[i] = 'Business'
    elif book_cats[i] == 'Health and Wellbeing':
        book_cats[i] = 'Health'
    elif book_cats[i] == 'Languages and Reference':
        book_cats[i] = 'Languages'
    elif book_cats[i] == 'Literature and Poetry':
        book_cats[i] = 'Literature'
    elif book_cats[i] == 'Medical and Nursing':
        book_cats[i] = 'Medical'
    elif book_cats[i] == 'Philosophy and Religion':
        book_cats[i] = 'Philosophy'
    elif book_cats[i] == 'Sports and Hobbies':
        book_cats[i] = 'Sports'
    elif book_cats[i] == 'Stage and Screen':
        book_cats[i] = 'Stage'
    elif book_cats[i] == 'Technology and Transport':
        book_cats[i] = 'Technology'
# Display the updated list
print(book_cats)

['Art and Design', 'Biography', 'Business, Economics and Law', "Children's", 'Computing', 'Crime', 'Education', 'Fiction', 'Food and Drink', 'Graphic Novels', 'Health', 'History', 'Home and Garden', 'Languages', 'Literature', 'Medical', 'Music', 'Philosophy', 'Politics', 'Science', 'SF, Fantasy & Horror', 'Social Sciences', 'Sports', 'Stage', 'Stationery and Gift', 'Technology', 'Teen and YA', 'Travel']


In [27]:
# Store in csv:

import csv

# Specify the file path for the CSV file
csv_file_path = "book_cats.csv"

# Open the CSV file in write mode
with open(csv_file_path, mode='w', newline='') as csv_file:
    # Create a CSV writer
    csv_writer = csv.writer(csv_file)

    # Write the header (if needed)
    # csv_writer.writerow(["Category Names"])

    # Write the data
    csv_writer.writerow(book_cats)

print(f"Category names have been exported to {csv_file_path}")


Category names have been exported to book_cats.csv


### Store best selling list per category

#### try 1 case

In [28]:
 # Construct URL for each category
book_cats_processed = [name.lower().replace(" ", "").replace("'", "") for name in book_cats]
category_name=book_cats_processed[0]
category_url = f"https://blackwells.co.uk/bookshop/category/_{category_name.strip().lower()}?offset=0&maxhits=60&sortValue=Bestselling"
category_response = requests.get(category_url)

if category_response.status_code == 200:
    category_soup = BeautifulSoup(category_response.content, "html.parser")
    best_sellers = category_soup.find_all("ul", class_='search-result__list')
    
    for ul_element in best_sellers:
        li_elements = ul_element.find_all("li")
        recommended_list = []
        
        # Loop through each 'li' element
        for item in li_elements[:50]:
            # Find the 'h4' element
            h4_element = item.find("h4")
            if h4_element:
                
                # Find the 'a' element with class 'product-name' inside 'h4'
                product_name_element = h4_element.find("a", class_="product-name")
                product_name = product_name_element.text.strip() if product_name_element else "N/A"
                
                # Find the 'div' element with class 'product-picture'
                product_picture_element = item.find("div", class_="product-picture")
                # Find the 'img' element inside 'product-picture'
                img_element = product_picture_element.find("img") if product_picture_element else None
                product_image = img_element["src"] if img_element and "src" in img_element.attrs else "N/A"
                
                # Find the 'p' element with class 'product-author'
                author_element = item.find("p", class_="product-author")
                author = author_element.text.strip() if author_element else "N/A"
 
                # Find the 'li' element with class 'product-price--current'
                price_element = item.find("li", class_="product-price--current")
                price = price_element.text.strip() if price_element else "N/A"
                
                # Append the extracted data to the recommended_list
                recommended_list.append({"Category":category_name,"Image": product_image, "Title": product_name, "Author": author, "Price": price})
        
        # Print or use the recommended_list here
        print(pd.DataFrame(recommended_list))
        
else:
    print(f"Failed to fetch data for category: {category_name}")
print(len(recommended_list))

        Category                              Image  \
0   artanddesign  /jacket/200x200/9781855145870.jpg   
1   artanddesign  /jacket/200x200/9781788842389.jpg   
2   artanddesign  /jacket/200x200/9781350143111.jpg   
3   artanddesign  /jacket/200x200/9780770435219.jpg   
4   artanddesign  /jacket/200x200/9781032324142.jpg   
5   artanddesign  /jacket/200x200/9780760382806.jpg   
6   artanddesign  /jacket/200x200/9781849947671.jpg   
7   artanddesign  /jacket/200x200/9781912235575.jpg   
8   artanddesign  /jacket/200x200/9781523519101.jpg   
9   artanddesign  /jacket/200x200/9783836507462.jpg   
10  artanddesign  /jacket/200x200/9781914236228.jpg   
11  artanddesign  /jacket/200x200/9780141995311.jpg   
12  artanddesign  /jacket/200x200/9781858947075.jpg   
13  artanddesign  /jacket/200x200/9780136746911.jpg   
14  artanddesign  /jacket/200x200/9781781577820.jpg   
15  artanddesign  /jacket/200x200/9784756253941.jpg   
16  artanddesign  /jacket/200x200/9781907896972.jpg   
17  artand

### Loop through all category

In [29]:
 # Construct URL for each category
#book_cats_processed = [name.lower().replace(" ", "") for name in book_cats]
all_results=[]
for category_name in book_cats:
    if category_name == "SF, Fantasy & Horror":
        category_processed = "sf"
    elif category_name == "Social Sciences":
        category_processed = "socsci"
    else:
        category_processed = category_name.lower().replace(" ", "").replace("'", "")
        category_url = f"https://blackwells.co.uk/bookshop/category/_{category_processed}?offset=0&maxhits=60&sortValue=Bestselling"
        category_response = requests.get(category_url)

        if category_response.status_code == 200:
            category_soup = BeautifulSoup(category_response.content, "html.parser")
            best_sellers = category_soup.find_all("ul", class_='search-result__list')

            recommended_list = []

            # Loop through each 'li' element
            for ul_element in best_sellers:
                li_elements = ul_element.find_all("li")
                
                for item in li_elements[:0]:
                    # Find the 'h4' element
                    h4_element = item.find("h4")
                    if h4_element:
                        # Find the 'a' element with class 'product-name' inside 'h4'
                        product_name_element = h4_element.find("a", class_="product-name")
                        product_name = product_name_element.text.strip() if product_name_element else "N/A"

                        # Find the 'div' element with class 'product-picture'
                        product_picture_element = item.find("div", class_="product-picture")
                        # Find the 'img' element inside 'product-picture'
                        img_element = product_picture_element.find("img") if product_picture_element else None
                        product_image = "https://blackwells.co.uk/"+img_element["src"] if img_element and "src" in img_element.attrs else "N/A"

                        # Find the 'p' element with class 'product-author'
                        author_element = item.find("p", class_="product-author")
                        author = author_element.text.strip() if author_element else "N/A"

                        # Find the 'li' element with class 'product-price--current'
                        price_element = item.find("li", class_="product-price--current")
                        price = price_element.text.strip() if price_element else "N/A"

                        # Append the extracted data to the recommended_list
                        recommended_list.append({"Category": category_name, "Image": product_image, "Title": product_name, "Author": author, "Price": price})

            # Print or use the recommended_list here
            all_results.extend(recommended_list)
            print(pd.DataFrame(all_results))
        else:
            print(f"Failed to fetch data for category: {category_name}")

all_bestsellers=pd.DataFrame(all_results)


          Category                                              Image  \
0   Art and Design  https://blackwells.co.uk//jacket/200x200/97818...   
1   Art and Design  https://blackwells.co.uk//jacket/200x200/97817...   
2   Art and Design  https://blackwells.co.uk//jacket/200x200/97813...   
3   Art and Design  https://blackwells.co.uk//jacket/200x200/97807...   
4   Art and Design  https://blackwells.co.uk//jacket/200x200/97810...   
5   Art and Design  https://blackwells.co.uk//jacket/200x200/97807...   
6   Art and Design  https://blackwells.co.uk//jacket/200x200/97818...   
7   Art and Design  https://blackwells.co.uk//jacket/200x200/97819...   
8   Art and Design  https://blackwells.co.uk//jacket/200x200/97815...   
9   Art and Design  https://blackwells.co.uk//jacket/200x200/97838...   
10  Art and Design  https://blackwells.co.uk//jacket/200x200/97819...   
11  Art and Design  https://blackwells.co.uk//jacket/200x200/97801...   
12  Art and Design  https://blackwells.co.uk//jacke

          Category                                              Image  \
0   Art and Design  https://blackwells.co.uk//jacket/200x200/97818...   
1   Art and Design  https://blackwells.co.uk//jacket/200x200/97817...   
2   Art and Design  https://blackwells.co.uk//jacket/200x200/97813...   
3   Art and Design  https://blackwells.co.uk//jacket/200x200/97807...   
4   Art and Design  https://blackwells.co.uk//jacket/200x200/97810...   
..             ...                                                ...   
64      Children's  https://blackwells.co.uk//jacket/200x200/97814...   
65      Children's  https://blackwells.co.uk//jacket/200x200/97814...   
66      Children's  https://blackwells.co.uk//jacket/200x200/97800...   
67      Children's  https://blackwells.co.uk//jacket/200x200/97803...   
68      Children's  https://blackwells.co.uk//jacket/200x200/97805...   

                                 Title                   Author   Price  
0   David Hockney - Normandy Portra...  David Hoc

           Category                                              Image  \
0    Art and Design  https://blackwells.co.uk//jacket/200x200/97818...   
1    Art and Design  https://blackwells.co.uk//jacket/200x200/97817...   
2    Art and Design  https://blackwells.co.uk//jacket/200x200/97813...   
3    Art and Design  https://blackwells.co.uk//jacket/200x200/97807...   
4    Art and Design  https://blackwells.co.uk//jacket/200x200/97810...   
..              ...                                                ...   
171  Food and Drink  https://blackwells.co.uk//jacket/200x200/97816...   
172  Food and Drink  https://blackwells.co.uk//jacket/200x200/97815...   
173  Food and Drink  https://blackwells.co.uk//jacket/200x200/97816...   
174  Food and Drink  https://blackwells.co.uk//jacket/200x200/97807...   
175  Food and Drink  https://blackwells.co.uk//jacket/200x200/97805...   

                                  Title                   Author   Price  
0    David Hockney - Normandy Portra

           Category                                              Image  \
0    Art and Design  https://blackwells.co.uk//jacket/200x200/97818...   
1    Art and Design  https://blackwells.co.uk//jacket/200x200/97817...   
2    Art and Design  https://blackwells.co.uk//jacket/200x200/97813...   
3    Art and Design  https://blackwells.co.uk//jacket/200x200/97807...   
4    Art and Design  https://blackwells.co.uk//jacket/200x200/97810...   
..              ...                                                ...   
278       Languages  https://blackwells.co.uk//jacket/200x200/97800...   
279       Languages  https://blackwells.co.uk//jacket/200x200/97815...   
280       Languages  https://blackwells.co.uk//jacket/200x200/97802...   
281       Languages  https://blackwells.co.uk//jacket/200x200/97802...   
282       Languages  https://blackwells.co.uk//jacket/200x200/97812...   

                                  Title                   Author   Price  
0    David Hockney - Normandy Portra

           Category                                              Image  \
0    Art and Design  https://blackwells.co.uk//jacket/200x200/97818...   
1    Art and Design  https://blackwells.co.uk//jacket/200x200/97817...   
2    Art and Design  https://blackwells.co.uk//jacket/200x200/97813...   
3    Art and Design  https://blackwells.co.uk//jacket/200x200/97807...   
4    Art and Design  https://blackwells.co.uk//jacket/200x200/97810...   
..              ...                                                ...   
396        Politics  https://blackwells.co.uk//jacket/200x200/97801...   
397        Politics  https://blackwells.co.uk//jacket/200x200/97814...   
398        Politics  https://blackwells.co.uk//jacket/200x200/97818...   
399        Politics  https://blackwells.co.uk//jacket/200x200/97816...   
400        Politics  https://blackwells.co.uk//jacket/200x200/97815...   

                                  Title                   Author   Price  
0    David Hockney - Normandy Portra

           Category                                              Image  \
0    Art and Design  https://blackwells.co.uk//jacket/200x200/97818...   
1    Art and Design  https://blackwells.co.uk//jacket/200x200/97817...   
2    Art and Design  https://blackwells.co.uk//jacket/200x200/97813...   
3    Art and Design  https://blackwells.co.uk//jacket/200x200/97807...   
4    Art and Design  https://blackwells.co.uk//jacket/200x200/97810...   
..              ...                                                ...   
506      Technology  https://blackwells.co.uk//jacket/200x200/97817...   
507      Technology  https://blackwells.co.uk//jacket/200x200/97813...   
508      Technology  https://blackwells.co.uk//jacket/200x200/97814...   
509      Technology  https://blackwells.co.uk//jacket/200x200/97818...   
510      Technology  https://blackwells.co.uk//jacket/200x200/97811...   

                                  Title                   Author    Price  
0    David Hockney - Normandy Portr

In [30]:
all_bestsellers=all_bestsellers[all_bestsellers['Price']!= "N/A"]
all_bestsellers['Price'] = all_bestsellers['Price'].str.replace('€', '').str.replace(',', '.').astype(float)

display(all_bestsellers)

,Category,Image,Title,Author,Price
0,Art and Design,https://blackwells.co.uk//jacket/200x200/97818...,David Hockney - Normandy Portra...,"David Hockney, Natio...",22.15
1,Art and Design,https://blackwells.co.uk//jacket/200x200/97817...,The Rolling Stones,Acc Art Books Ltd (C...,63.87
2,Art and Design,https://blackwells.co.uk//jacket/200x200/97813...,The Intersection of Fashion and...,Kate Annett-Hitchcock,26.85
3,Art and Design,https://blackwells.co.uk//jacket/200x200/97807...,Urban Watercolor Sketching,Felix Scheinberger,21.03
4,Art and Design,https://blackwells.co.uk//jacket/200x200/97810...,Experiencing Accents,Philip Thompson (aut...,38.37
...,...,...,...,...,...
552,Travel,https://blackwells.co.uk//jacket/200x200/97800...,Bangkok Days,Lawrence Osborne,15.15
553,Travel,https://blackwells.co.uk//jacket/200x200/97816...,Pirate Women,Laura Sook Duncombe,20.26
554,Travel,https://blackwells.co.uk//jacket/200x200/97806...,Britain's Birds,"Rob Hume, Rob Still,...",25.36
555,Travel,https://blackwells.co.uk//jacket/200x200/97819...,Lost Lanes Central England,Jack Thurston,19.92


In [31]:
all_bestsellers.to_csv('all_bestsellers.csv', index=False)

In [8]:
len(all_results)

557

In [53]:
book_cats

['Art and Design',
 'Biography',
 'Business, Economics and Law',
 "Children's",
 'Computing',
 'Crime',
 'Education',
 'Fiction',
 'Food and Drink',
 'Graphic Novels',
 'Health',
 'History',
 'Home and Garden',
 'Languages',
 'Literature',
 'Medical',
 'Music',
 'Philosophy',
 'Politics',
 'Science',
 'SF, Fantasy & Horror',
 'Social Sciences',
 'Sports',
 'Stage',
 'Stationery and Gift',
 'Technology',
 'Teen and YA',
 'Travel']

In [32]:
all_bestsellers.dtypes

Category     object
Image        object
Title        object
Author       object
Price       float64
dtype: object

In [33]:
all_bestsellers["Category"].value_counts()

Children's             25
Teen and YA            24
Graphic Novels         24
Literature             24
Philosophy             23
Education              23
Medical                23
Languages              22
Crime                  22
Fiction                22
Technology             22
Stationery and Gift    21
Stage                  21
Sports                 21
Politics               21
Art and Design         21
Music                  21
Biography              21
History                21
Travel                 21
Science                20
Health                 20
Computing              20
Food and Drink         19
Home and Garden        18
Name: Category, dtype: int64